In [2]:
import numpy as np
import glob
import xarray as xr
import netCDF4

import matplotlib.pyplot as plt

# CPOL convenience functions
import CPOL_func as cf


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/home/563/esh563/pyart/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/home/563/esh563/pyart/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [1]:
from caine_func import wrf_radar_npy_to_nc

In [ ]:
wrf_radar_npy_to_nc()

In [5]:
fn_CPOL = cf.CPOL_files_from_datetime_list(
    np.arange(np.datetime64('2006-02-10 12:00'), 
              np.datetime64('2006-02-10 12:10'), 
              np.timedelta64(10, 'm'))
)[0]

Gathering files.


In [26]:
CPOL = netCDF4.Dataset(('/g/data/rr5/CPOL_radar/CPOL_level_1b/' 
                        + 'GRIDDED/GRID_150km_2500m/2006/20060210/'
                        + 'CPOL_20060210_1200_GRIDS_2500m.nc'), mode='r')

In [3]:
CPOL.variables['time']

NameError: name 'CPOL' is not defined

In [7]:
CPOL = xr.open_dataset(('/g/data/rr5/CPOL_radar/CPOL_level_1b/' 
                       + 'GRIDDED/GRID_150km_2500m/2006/20060210/'
                       + 'CPOL_20060210_1200_GRIDS_2500m.nc'))

In [11]:
CPOL.reflectivity.encoding

{'zlib': True,
 'shuffle': True,
 'complevel': 4,
 'fletcher32': False,
 'contiguous': False,
 'chunksizes': (1, 41, 117, 117),
 'source': '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_150km_2500m/2006/20060210/CPOL_20060210_1200_GRIDS_2500m.nc',
 'original_shape': (1, 41, 117, 117),
 'dtype': dtype('float32'),
 '_FillValue': -9999.0}

In [3]:
test2 = xr.open_dataset('/g/data/w40/esh563/d04.dir/wrfout_d04_2006-02-09_10:00:00.nc.gz')

In [10]:
test2.PB[:,0,0,1:3]

<xarray.DataArray 'PB' (Time: 3, west_east: 2)>
array([[99621.97, 99568.65],
       [99621.97, 99568.65],
       [99621.97, 99568.65]], dtype=float32)
Coordinates:
    XLAT     (Time, west_east) float32 ...
    XLONG    (Time, west_east) float32 ...
Dimensions without coordinates: Time, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  BASE STATE PRESSURE
    units:        Pa
    stagger:      

In [89]:
ds['origin_latitude'] = ('time', CPOL.origin_latitude.values)
ds.origin_latitude.attrs = CPOL.origin_latitude.attrs

In [91]:
ds.origin_latitude

<xarray.DataArray 'origin_latitude' (time: 1)>
array([-12.249167])
Coordinates:
  * time     (time) datetime64[ns] 2006-02-08T12:00:00
Attributes:
    long_name:      Latitude at grid origin
    units:          degrees_north
    standard_name:  latitude
    valid_min:      -90.0
    valid_max:      90.0

In [12]:
fn = sorted(glob.glob('/g/data/w40/esh563/lind04/dbz/*.npy'))

In [13]:
fn[0][:-4]

'/g/data/w40/esh563/lind04/dbz/2006-02-08_12:00:00alllevels_zlib'

In [50]:
datetimes = np.arange(np.datetime64('2006-02-08 12:00:00Z'), 
            np.datetime64('2006-02-13 12:10:00Z'), 
            np.timedelta64(10, 'm'))

In [44]:
print(len(datetimes))
print(len(fn))

721
721


In [45]:
z = np.arange(500, 500*41, 500, dtype=float)
x = np.arange(0, 1250*241, 1250, dtype=float)
y = np.arange(0, 1250*241, 1250, dtype=float)

# for i in range(len(fn[0:1])):
#     data = np.load(fn[0])

In [46]:
attr_list = [eval('CPOL.{}.attrs'.format(c)) for c in ['x', 'y', 'z']]

AttributeError: NetCDF: Attribute not found

In [47]:
data.shape
240*3
del ds

NameError: name 'data' is not defined

'2006-02-08T12:00:00Z'

In [11]:
i=0
data = np.load(fn[i])
data[data==-20]=np.nan
data = np.expand_dims(data, 0)
da = xr.DataArray(data, dims=('time', 'z', 'x', 'y'), 
                  coords={'time': np.array([datetimes[i]]), 
                          'z':z, 'x':x, 'y':y})
da.time.encoding['units'] = 'seconds since  ' + str(datetimes[i]) + 'Z'
da.time.attrs['standard_name'] = 'time'
da.time.attrs['calendar'] = 'gregorian'

da.coords['x'] -= da.x.mean()
da.coords['y'] -= da.y.mean()

da.x.attrs = CPOL.x.attrs
da.y.attrs = CPOL.y.attrs
da.z.attrs = CPOL.z.attrs

ds = xr.Dataset({'reflectivity':da})
ds.reflectivity.attrs = CPOL.reflectivity.attrs

ds = ds.coarsen(x=2, boundary='trim', side='left').mean()
ds = ds.coarsen(y=2, boundary='trim', side='left').mean()

# Copy metadata from CPOL dataset
var_list = ['projection', 'ProjectionCoordinateSystem', 
            'radar_latitude', 'radar_longitude', 'radar_altitude']
for var in var_list: 
    ds[var] = CPOL[var]
    
var_list = ['origin_latitude', 'origin_longitude', 'origin_altitude']
for var in var_list:
    ds[var] = ('time', CPOL[var].values)
    ds[var].attrs = CPOL[var].attrs

ds['radar_time'] = ('nradar', [datetimes[i]])
ds.radar_time.attrs = CPOL.radar_time.attrs

NameError: name 'fn' is not defined

In [13]:
CPOL.origin_altitude

<xarray.DataArray 'origin_altitude' (time: 1)>
array([50.])
Coordinates:
  * time     (time) datetime64[ns] 2006-02-10T12:00:09
Attributes:
    long_name:      Altitude at grid origin
    units:          m
    standard_name:  altitude

In [23]:
var_list = ['origin_latitude', 'origin_longitude', 'origin_altitude']
for var in var_list:
    ds[var] = ('time', CPOL[var].values)
    ds[var].attrs = CPOL[var].attrs

In [29]:
ds['radar_time'] = ('nradar', [datetimes[0]])
ds.radar_time.attrs = CPOL.radar_time.attrs

In [10]:
test.reflectivity[0,0][:,50]

<xarray.DataArray 'reflectivity' (x: 120)>
array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan])
Coordinates:
    time     datetime64[ns] 2006-02-08T12:00:00
    z        float64 500.0
  * x        (x) float64 -1.494e+05 -1.469e+05 ... 1.456e+05 1.481e+05
    y        float64 -2.438e+04
Attributes:
    long_name:      Corrected reflectivity
    units:          dBZ
    standard_name:  cor

In [12]:
test = xr.open_dataset('/g/data/w40/esh563/lind04/dbz/2006-02-08_12:00:00alllevels_zlib.nc')

In [18]:
test.reflectivity.encoding['_FillValue'] = -9999.0
test.origin_altitude.encoding

{'zlib': False,
 'shuffle': False,
 'complevel': 0,
 'fletcher32': False,
 'contiguous': True,
 'chunksizes': None,
 'source': '/g/data/w40/esh563/lind04/dbz/2006-02-08_12:00:00alllevels_zlib.nc',
 'original_shape': (1,),
 'dtype': dtype('float64'),
 '_FillValue': nan}

In [35]:
test2 = netCDF4.Dataset('/g/data/w40/esh563/lind04/dbz/2006-02-08_12:00:00alllevels_zlib.nc', 'r')

In [27]:
test2

<xarray.Dataset>
Dimensions:              (Time: 3, bottom_top: 63, bottom_top_stag: 64, south_north: 486, south_north_stag: 487, west_east: 717, west_east_stag: 718)
Coordinates:
    XLAT                 (Time, south_north, west_east) float32 ...
    XLONG                (Time, south_north, west_east) float32 ...
    XLAT_U               (Time, south_north, west_east_stag) float32 ...
    XLONG_U              (Time, south_north, west_east_stag) float32 ...
    XLAT_V               (Time, south_north_stag, west_east) float32 ...
    XLONG_V              (Time, south_north_stag, west_east) float32 ...
Dimensions without coordinates: Time, bottom_top, bottom_top_stag, south_north, south_north_stag, west_east, west_east_stag
Data variables:
    Times                (Time) |S19 ...
    U                    (Time, bottom_top, south_north, west_east_stag) float32 ...
    V                    (Time, bottom_top, south_north_stag, west_east) float32 ...
    W                    (Time, bottom_to

In [26]:
((test2.PH+test2.PHB)/9.80665 - test2.HGT)[0].mean(axis=1).mean(axis=1)

<xarray.DataArray (bottom_top_stag: 64)>
array([1.3219908e-02, 4.4793285e+01, 8.9712303e+01, 1.3479427e+02,
       1.8004054e+02, 2.2545380e+02, 2.7103815e+02, 3.1679684e+02,
       3.6273050e+02, 4.0884189e+02, 4.5513358e+02, 5.4827319e+02,
       6.4218195e+02, 7.3689020e+02, 8.3242444e+02, 9.2880554e+02,
       1.0260477e+03, 1.1241650e+03, 1.2231710e+03, 1.3230847e+03,
       1.4239263e+03, 1.5257188e+03, 1.7321791e+03, 1.9426493e+03,
       2.1573389e+03, 2.3764587e+03, 2.6002295e+03, 2.8288596e+03,
       3.0625569e+03, 3.3015271e+03, 3.5460278e+03, 3.7963989e+03,
       4.0530447e+03, 4.3164565e+03, 4.5871494e+03, 4.8656387e+03,
       5.1525542e+03, 5.4485522e+03, 5.7542603e+03, 6.0703291e+03,
       6.3974321e+03, 6.7363555e+03, 7.0879751e+03, 7.4531763e+03,
       7.8328564e+03, 8.2281035e+03, 8.6401787e+03, 9.0708594e+03,
       9.5223105e+03, 9.9966865e+03, 1.0496958e+04, 1.1026672e+04,
       1.1589865e+04, 1.2191847e+04, 1.2839143e+04, 1.3539489e+04,
       1.4304395e+04,

In [12]:
test3 = np.load('/g/data/w40/esh563/lind04/dbz/2006-02-08_12:00:00alllevels_zlib.npy')

In [37]:
test2.variables['time']

<class 'netCDF4._netCDF4.Variable'>
int64 time(time)
    units: days since 2006-02-08 12:00:00
    calendar: proleptic_gregorian
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of -9223372036854775806 used

In [91]:
ds.groupby_bins('x', np.arange(0, 302500, 2500), right=False, labels=x_bin_labels).mean()

NameError: name 'x_bin_labels' is not defined

In [ ]:
# Collect CPOL attributes
attrs = CPOL.attrs

In [13]:
ds.time

NameError: name 'ds' is not defined

In [5]:
data_list = [np.load(fn[i]) for i in range(len(fn))]

In [6]:
data = np.stack(data_list)

In [7]:
data.shape

(72, 40, 241, 241)